## Day 16: Chronal Classification

https://adventofcode.com/2018/day/16

### Part 1

In [4]:
from pyrsistent import pvector
import operator as op
from parse import parse


def opr(f):
    def fr(registers, a, b, c):
        return registers.set(c, f(registers[a], registers[b]))
    
    return fr


def opi(f):
    def fi(registers, a, b, c):
        return registers.set(c, f(registers[a], b))
    
    return fi


addr = opr(op.add)
addi = opi(op.add)
mulr = opr(op.mul)
muli = opi(op.mul)

banr = opr(op.and_)
bani = opi(op.and_)
borr = opr(op.or_)
bori = opi(op.or_)

def setr(r, a, b, c):
    return r.set(c, r[a])
def seti(r, a, b, c):
    return r.set(c, a)

def gtir(r, a, b, c):
    return r.set(c, 1 if a > r[b] else 0)
def gtri(r, a, b, c):
    return r.set(c, 1 if r[a] > b else 0)
def gtrr(r, a, b, c):
    return r.set(c, 1 if r[a] > r[b] else 0)

def eqir(r, a, b, c):
    return r.set(c, 1 if a == r[b] else 0)
def eqri(r, a, b, c):
    return r.set(c, 1 if r[a] == b else 0)
def eqrr(r, a, b, c):
    return r.set(c, 1 if r[a] == r[b] else 0)


opcodes = pvector([addr, addi, mulr, muli, banr, bani, 
                   borr, bori, setr, seti, gtir, gtri,
                   gtrr, eqir, eqri, eqrr])


def matches(before, after, a, b, c, opcode):
    try:
        return opcode(before, a, b, c) == after
    except:
        return False

def num_matches(before, after, a, b, c):
    return sum(1 if matches(before, after, a, b, c, opcode) else 0
               for opcode in opcodes)


assert num_matches(pvector([3, 2, 1, 1]), 
                   pvector([3, 2, 2, 1]), 2, 1, 2) == 3

In [21]:
from itertools import islice

lines = open('input_1', 'r').readlines()

n = 0

for i in range(0, len(lines), 4):
    before = pvector(parse('Before: [{:d}, {:d}, {:d}, {:d}]', lines[i]).fixed)
    _, a, b, c = parse('{:d} {:d} {:d} {:d}', lines[i + 1]).fixed
    after = pvector(parse('After:  [{:d}, {:d}, {:d}, {:d}]', lines[i + 2]).fixed)
    if num_matches(before, after, a, b, c) >= 3:
        n += 1
        
n

612

### Part 2

